# Autómatas: AFD, AFN y AFN-Lambda

In [1]:
# Import libraries.
import numbers
import math
import numpy as np
import matplotlib.pyplot as plt

#### 1. Clase Autómata

---


Lorem Ipsum es simplemente el texto de relleno de las imprentas y archivos de texto. Lorem Ipsum ha sido el texto de relleno estándar de las industrias desde el año 1500, cuando un impresor (N. del T. persona que se dedica a la imprenta) desconocido usó una galería de textos y los mezcló de tal manera que logró hacer un libro de textos especimen. 

<img src = "https://upload.wikimedia.org/wikipedia/commons/thumb/9/9d/DFAexample.svg/400px-DFAexample.svg.png" width = "200px">

In [340]:
class Automata:
    
    def __init__(self, language = [], initial_state = 0, states = [], valid_states = [], delta = []):
        
        self.initial_state = initial_state
        self.language = language
        self.valid_states = valid_states
        self.adjMatrix = self.generarMatrizAdyacencia(states, delta)
        #self.type = self.determinarTipoAutomata()
    
    def generarMatrizAdyacencia(self, states, delta):
        
        # Create matrix object.
        adjMatrix = list()
        
        # Number of layers of the adyacency matrix (depth).
        n = len(states)
        for i in range(len(self.language)):
              adjMatrix.append(np.zeros((n, n)))
                
        # Delta codifies the edges between nodes.
        for edge in delta:
            
            # Example: edge = ["q0", "q1", "a"]
            # Get states number.
            first_state = states.index(edge[0])
            second_state = states.index(edge[1])
            
            # Get instruction.
            instruction = self.language.index(edge[2])
            
            # Add new edge.
            adjMatrix[instruction][first_state][second_state] = 1
            
        # Return adyacency matrix.
        return adjMatrix

    # Utility functions.
    def obtenerNumerosEstadosAccesibles(self, current_state, instruction):
        layer = self.adjMatrix[instruction]
        bridges = layer[current_state]
        numbers = list()
        for b in range(len(bridges)):
            if bridges[b] == 1: numbers.append(b)
        return numbers
    
    def corroborarAceptacion(self, acceptanceList):
        if 1 in acceptanceList: return 1
        else: return 0   
    
    """
    def determinarTipoAutomata(self): 
        
      
        Type      Code
        ---------------
        AFD       0
        AFN       1
        AFN-e     2
    
        if "e" in self.language: return 2
        else: 
            deterministic = True
            for instruction in self.language: 
                layer = self.adjMatrix[self.language.index(instruction)]
                for row_index in range(len(layer)):
                    lista = self.obtenerNumerosEstadosAccesibles(
                        row_index, 
                        self.language.index(instruction)
                    )
                    if len(lista) > 1: 
                        return 1
                    if len(lista) == 0:
                        deterministic = False
        if deterministic: 
            return 0
        else: 
            raise Exception("Todas las transiciones tienen que estar definidas!")
    """
        
    def _obtenerSecuenciaEstadoInstruccion(self, cadena, recorrido):
        secuencia = list()
        
        #print("")
        #print("Cadena: ", cadena)
        #print("Recorrido: ", recorrido)
        #print("")
        
        for i in range(len(recorrido)): 
            try:
                secuencia.append((recorrido[i], cadena[i]))
            except:
                #if i != len(recorrido) - 1:
                #    secuencia.append((recorrido[i], "e"))
                #else:
                secuencia.append((recorrido[i], ""))
        return secuencia 
    
    def _validChain(self, trayectorias):
        for trayectoria in trayectorias: 
            if trayectoria[1] == 1:
                return True
        return False
    
    def _getTrayectoriesInfo(self, trayectorias):
        
        # Accepted.
        accepted = self._validChain(trayectorias)
        if accepted: accepted = "Si"
        else: accepted = "No"
        
        # Get info from trayectories.
        accepted_trayectories = list()
        rejected_trayectories = list()
        aborted_trayectories = list()
        
        # Classify by trayectory type.
        for trayectoria in trayectorias: 
            if trayectoria[1] == 1:
                accepted_trayectories.append(trayectoria)
            elif trayectoria[1] == 0:
                aborted_trayectories.append(trayectoria)
            elif trayectoria[1] == -1:
                rejected_trayectories.append(trayectoria)
                
        # Get shortest trayectory for each type.
        shortest_accepted = self._getShortestTrayectory(accepted_trayectories)
        shortest_aborted = self._getShortestTrayectory(aborted_trayectories)
        shortest_rejected = self._getShortestTrayectory(rejected_trayectories)
        
        # Compute trayectories features.
        
        # número de posibles procesamientos.
        total_trayectories = len(trayectorias) 
        total_trayectories_accepted = len(accepted_trayectories)
        total_trayectories_rejected = len(rejected_trayectories)
        total_trayectories_aborted = len(aborted_trayectories)
    
    
        return[shortest_accepted,  
               shortest_rejected, 
               shortest_aborted,
               total_trayectories, 
               total_trayectories_accepted,
               total_trayectories_aborted,
               total_trayectories_rejected,
               accepted
              ]
        
    """
    Input Example: [0110, 1, ab]
    Output Example: [0110, aab]
    """
    def _getShortestTrayectory(self, trayectorias):
        
        if len(trayectorias) > 0:
            
            #print("trayectorias es mayor a 0!!!")
            
            recorridos = list()
            for trayectoria in trayectorias: 
                recorrido = trayectoria[0]
                recorridos.append(recorrido)

            shortest_path_length = len(recorridos[0])
            shortest_path_index = 0

            for recorrido in recorridos: 
                if len(recorrido) < shortest_path_length: 
                    shortest_path_length = len(recorrido)
                    shortest_path_index = recorridos.index(recorrido)
            return trayectorias[shortest_path_index]
        return ["", "", ""]

In [341]:
delta = [
          ["q0", "q0", "a"], 
          ["q0", "q1", "b"], 
          ["q1", "q1", "b"], 
          ["q1", "q0", "a"], 
        ]

autom = Automata(language = ["a", "b"], states = ["q0", "q1"], delta = delta)

#### 2. Autómatas Deterministas.

In [342]:
class AFD(Automata):
    def __init__(self, language, initial_state, states, valid_states, delta): 
        super().__init__(language, initial_state, states, valid_states, delta)
        #if self.type != 0: raise Exception("Los argumentos ingresados, no corresponden a un autómata determinista.")
        
    def procesarCadena(self, cadena):
        print(self.__procesar(cadena)[1] == 1)
    
    def procesarCadenaConDetalles(self, cadena): 
        process = self.__procesar(cadena)
        print(process[1] == 1)
        print(self._obtenerSecuenciaEstadoInstruccion(cadena, process[0]))
    
    def procesarListaCadenas(self, listaCadenas, nombreArchivo, imprimirPantalla): 
        
        try:
            file = open(nombreArchivo, "w+")
        except: 
            file = open("resultadosAFD.txt", "w+")
            
        for cadena in listaCadenas: 
            
            process = self.__procesar(cadena)
            secuencia = self._obtenerSecuenciaEstadoInstruccion(
                cadena,    # cadena
                process[0] # recorrido
            )
            
            file.write(cadena + "\t")
            
            if imprimirPantalla: print(cadena, end = "\n" )
                
            for pareja in secuencia:
                if imprimirPantalla: print(str(pareja), end = "\t")
                file.write(str(pareja) + "\t")
                
            if process[1] == 1: aceptado = "Si"
            else: aceptado = "No"
            file.write(aceptado + "\n")
            
            if imprimirPantalla: 
                print("")
                print(aceptado, end = "\n")
        
        file.close()
        return "Archivo creado con éxito"

    def __procesar(self, cadena, current_state = 0):
        
        # If chain is not null.
        if cadena != "":
            
            # Get next state number given by instruction in the chain.
            newState = self.obtenerNumerosEstadosAccesibles(
                current_state, 
                self.language.index(cadena[0])   
            )
            #print("New State: ", newState)
            newState = newState[0]

            # Move to next state.
            memory = self.__procesar(cadena[1:], current_state = newState)
            memory[0] = str(current_state) + memory[0]
            return memory

        # If chain is null.
        else: 
            if current_state in self.valid_states:
                return [str(current_state), 1]    # Accepted.
            else:
                return [str(current_state), -1]   # Rejected.

In [343]:
language = ["a", "b"]
initial_state = 0
states = ["q0", "q1"]
delta = [
          ["q0", "q0", "a"], 
          ["q0", "q1", "b"], 
          ["q1", "q1", "b"], 
          ["q1", "q0", "a"], 
        ]
afd = AFD(language = language, 
          initial_state = initial_state, 
          valid_states = [0], 
          states = states, 
          delta = delta)

afd.procesarCadenaConDetalles("abaaaaababa")
print("")

afd.procesarListaCadenas(
    ["abaaaaabab", "aaabbba", "bbbbabaab"], 
    "resultadosAFD.txt", 
    False
)

True
[('0', 'a'), ('0', 'b'), ('1', 'a'), ('0', 'a'), ('0', 'a'), ('0', 'a'), ('0', 'a'), ('0', 'b'), ('1', 'a'), ('0', 'b'), ('1', 'a'), ('0', '')]



'Archivo creado con éxito'

#### 3. Autómatas No-Deterministas.

In [402]:
class AFN(Automata):
    def __init__(self, language, initial_state, states, valid_states, delta): 
        super().__init__(language, initial_state, states, valid_states, delta)
        #if self.type != 1: raise Exception("Los argumentos ingresados, no corresponden a un autómata no determinista.")
        
    def procesarCadena(self, cadena):
        trayectorias = self.__procesar(cadena)
        if self._validChain(trayectorias): print(True)
        else: print(False)
    
    def procesarCadenaConDetalles(self, cadena): 
        trayectorias = self.__procesar(cadena)
        print("Procesamientos que terminan en un estado de aceptación: ")
        print("")
        for trayectoria in trayectorias:
            if trayectoria[1] == 1:
                print("Procesamiento No. ", trayectorias.index(trayectoria))
                print(self._obtenerSecuenciaEstadoInstruccion(cadena, trayectoria[0]))
                print(" ")
                
    def procesarListaCadenas(self, listaCadenas, nombreArchivo, imprimirPantalla): 
        
        try:
            file = open(nombreArchivo, "w+")
        except: 
            file = open("resultadosAFN.txt", "w+")
            
        for cadena in listaCadenas: 
            
            if imprimirPantalla: 
                print("------Nueva cadena-----")
                print("")
                print(cadena, end = "\t")
            
            # Save chain on document.
            file.write(cadena + "\t")
            
            # Process chain.
            trayectorias = self.__procesar(cadena)
            
            # Get info of trayectories.
            info = self._getTrayectoriesInfo(trayectorias)
            
            secuencia = []
            
            #print("INFO:")
            #print(info)
            
            # If chain was accepted.
            if info[0][0] != "":
                secuencia = self._obtenerSecuenciaEstadoInstruccion(
                    cadena,
                    info[0][0]
                )
            else:
                # If chain was rejected.
                if info[1][0] != "":
                    secuencia = self._obtenerSecuenciaEstadoInstruccion(
                        cadena,
                        info[1][0]
                    )
                # If chain was aborted.
                else:
                    secuencia = self._obtenerSecuenciaEstadoInstruccion(
                        cadena, 
                        info[2][0]
                    )
                    
            # Add secuency to document.
            for pareja in secuencia:
                if imprimirPantalla: print(str(pareja), end = "\t")
                file.write(str(pareja) + "\t")
                
            if imprimirPantalla: 
                for element in info[3:]:
                    print(element, end = "\t")
                print("")
                print("")
                
            file.write(str(info[3]) + "\t") # Numero de posibles procesamientos.
            file.write(str(info[4]) + "\t") # Numero de procesamiento de aceptación.
            file.write(str(info[5]) + "\t") # Número de procesamientos abortados.
            file.write(str(info[6]) + "\t") # Número de procesamientos recahazados
            file.write(str(info[7]) + "\n") # Si o no
            
        file.close()
        return "Archivo creado con éxito"
    
    """
    -procesar.
    Output example: 
    [['00000', 0],
     ['000011', 1],
     ['00010', 0]]
    """
    def __procesar(self, cadena, current_state = 0):
              
        #print("Current State: ", current_state)
        #print("Cadena: ", cadena)
        #print("Cadena[1:]", cadena[1:])
        
        # If chain is not empty.
        if cadena != "":
            
            # Get next states numbers given by instruction in the chain.
            newStatesNumbers = self.obtenerNumerosEstadosAccesibles(
                current_state, 
                self.language.index(cadena[0])   
            )
                    
            # If there exist new states where to move.
            if newStatesNumbers != []:
                
                acceptance_list = list()
                for newState in newStatesNumbers:
                    result = self.__procesar(cadena[1:], newState)
                    for pair in result: 
                        acceptance_list.append(pair)
                for pair in acceptance_list: 
                    memory = pair[0] # Get trayectory.                    
                    pair[0] = str(current_state) + memory
                return acceptance_list
 
            # Aborted! Since there is no where to move.
            else: 
                return [[str(current_state), 0]]    # Aborted.       
        
        # If chain is empty.
        else: 
            if current_state in self.valid_states:
                return [[str(current_state), 1]]    # Accepted.
            else:
                return [[str(current_state), -1]]  # Rejected.

In [403]:
language = ["a", "b"]
initial_state = 0
states = ["q0", "q1", "q2"]
delta = [
          ["q0", "q0", "a"], 
          ["q0", "q1", "a"], 
          ["q1", "q1", "b"], 
          ["q1", "q0", "a"], 
          ["q1", "q2", "a"]
        ]
afn = AFN(language = language, 
          initial_state = initial_state, 
          valid_states = [0], 
          states = states, 
          delta = delta)

afn.procesarCadena("aba")
print("")
afn.procesarCadenaConDetalles("aba")

afn.procesarListaCadenas(
    ["aba", "aaab", "bbaaabba", "abbaaabba"], 
    "resultadosAFN.txt", 
    False
)

True

Procesamientos que terminan en un estado de aceptación: 

Procesamiento No.  1
[('0', 'a'), ('1', 'b'), ('1', 'a'), ('0', '')]
 


'Archivo creado con éxito'

#### 4. Autómatas con transiciones lambda.

In [404]:
class AFNLambda(Automata):
    def __init__(self, language, initial_state, states, valid_states, delta): 
        super().__init__(language, initial_state, states, valid_states, delta)
        
    def calcularLambdaClausura(self, current_state = 0):
        # Get next states numbers given by lambda.
        newLambdaStatesNumbers = self.obtenerNumerosEstadosAccesibles(
            current_state, 
            self.language.index("e")   
        )
        
        if len(newLambdaStatesNumbers) > 0: 
            lambdaLista = list()
            
            # Add current state to the list.
            lambdaLista.append(current_state)
            for lambdaStateNum in newLambdaStatesNumbers: 
                clausura = self.calcularLambdaClausura(lambdaStateNum)
                for estado in clausura: 
                    lambdaLista.append(estado)
                return lambdaLista
        else:    
            return [current_state]
    
    def calcularLambdaClausuras(self, lista_estados):
        clausuras = list()
        for current_state in lista_estados: 
            clausuras.append(self.calcularLambdaClausura(current_state))
        return clausuras
    
    def procesarCadena(self, cadena):
        trayectorias = self.procesar(cadena)
        if self._validChain(trayectorias): print(True)
        else: print(False)
    
    def procesarCadenaConDetalles(self, cadena): 
        trayectorias = self.procesar(cadena)
        print("Procesamientos que terminan en un estado de aceptación: ")
        print("")
        for trayectoria in trayectorias:
            if trayectoria[1] == 1:
                print("Procesamiento No. ", trayectorias.index(trayectoria))
                memoria_instrucciones = trayectoria[2]
                memoria_recorrido = trayectoria[0]
                print(self._obtenerSecuenciaEstadoInstruccion(
                    memoria_instrucciones, 
                    memoria_recorrido
                ))
                print(" ")

    def computarTodosLosProcesamientos(self, cadena): 
        return ""
    
    def procesarListaCadenas(self, listaCadenas, nombreArchivo, imprimirPantalla): 
        try:
            file = open(nombreArchivo, "w+")
        except: 
            file = open("resultadosAFNLambda.txt", "w+")
            
        for cadena in listaCadenas: 
            
            if imprimirPantalla: 
                print("------Nueva cadena-----")
                print("")
                print(cadena, end = "\t")
            
            # Save chain on document.
            file.write(cadena + "\t")
            
            # Process chain.
            trayectorias = self.procesar(cadena)
            
            # Get info of trayectories.
            #print("Trayectorias: ")
            #print(trayectorias)
            info = self._getTrayectoriesInfo(trayectorias)
            print(info)
            
            secuencia = []
            
            # If chain was accepted.
            if info[0][0] != "":
                secuencia = self._obtenerSecuenciaEstadoInstruccion(
                    info[0][2],
                    info[0][0]
                )
            else:
                # If chain was rejected.
                if info[1][0] != "":
                    secuencia = self._obtenerSecuenciaEstadoInstruccion(
                        info[1][2],
                        info[1][0]
                    )
                # If chain was aborted.
                else:
                    secuencia = self._obtenerSecuenciaEstadoInstruccion(
                        info[2][2], 
                        info[2][0]
                    )
                    
            # Add secuency to document.
            for pareja in secuencia:
                if imprimirPantalla: print(str(pareja), end = "\t")
                file.write(str(pareja) + "\t")
                
            if imprimirPantalla: 
                for element in info[3:]:
                    print(element, end = "\t")
                print("")
                print("")
                
            file.write(str(info[3]) + "\t") # Numero de posibles procesamientos.
            file.write(str(info[4]) + "\t") # Numero de procesamiento de aceptación.
            file.write(str(info[5]) + "\t") # Número de procesamientos abortados.
            file.write(str(info[6]) + "\t") # Número de procesamientos recahazados
            file.write(str(info[7]) + "\n") # Si o no
            
        file.close()
        return "Archivo creado con éxito"
    
    def procesar(self, cadena, current_state = 0, ultima_instruccion = ""):
        
        #print("----")
        #print("Current State: ", current_state, " - Cadena: ", cadena)
        
        # Get new lambda states.
        newLambdaStatesNumbers = self.obtenerNumerosEstadosAccesibles(
            current_state, 
            self.language.index("e")   
        )
        
        #print("New Lambda States: ", newLambdaStatesNumbers)
       
        
        # If chain is not empty.
        if cadena != "" or newLambdaStatesNumbers != []:
            
            acceptance_list = list()
            abortado = 0
            """
            Si la cadena es diferente de vacio, siga buscando transiciones
            diferentes a Lambda
            """
            # Get next states numbers given by instruction in the chain.
            if cadena != "":
                newStatesNumbers = self.obtenerNumerosEstadosAccesibles(
                    current_state, 
                    self.language.index(cadena[0])   
                )
                    
                # If there exist new states where to move.
                if newStatesNumbers != []:
                    # Normal transitions.
                    for newState in newStatesNumbers:
                        result = self.procesar(cadena[1:], newState, cadena[0])
                        for pair in result: 
                            acceptance_list.append(pair)
                # There is no where to move using "normal" transitions.
                else: abortado += 1
                        
            """
            Si existen transicion lambda disponibles, transite hacia ellas
            """
            # Lambda transitions.
            if newLambdaStatesNumbers != []:
                for newLambdaState in newLambdaStatesNumbers: 
                    result = self.procesar(cadena, newLambdaState, "e")
                    for pair in result: 
                        acceptance_list.append(pair)
            # There is no where to move using lambda transitions.
            else: abortado += 1

            # Aborted! Since there is no where to move.
            if abortado == 2: return [[str(current_state), 0, ultima_instruccion]]    # Aborted.  
            else:
                """
                Añada la memoria del recorrido.
                """
                # Añadir memoria de estados e instrucciones.
                for pair in acceptance_list: 
                    #print("Pair: ", pair)
                    
                    # Actualizar la memoria de estados por los cuales
                    # transitó el autómata.
                    memory = pair[0] # Get trayectory.                    
                    pair[0] = str(current_state) + memory
                    
                    # Actualizar la lista de instrucciones que ejecutó
                    # el automata.
                    instructions = pair[2]
                    pair[2] = str(ultima_instruccion) + instructions

                return acceptance_list
        
        # If chain is empty.
        else:             
            if current_state in self.valid_states:
                return [[str(current_state), 1, ultima_instruccion]]    # Accepted.
            
            else:
                return [[str(current_state), -1, ultima_instruccion]]  # Rejected.

In [405]:
language = ["a", "b", "e"]
initial_state = 0
states = ["q0", "q1", "q2"]
delta = [
    ["q0", "q0", "a"], 
    ["q0", "q1", "b"], 
    ["q1", "q1", "b"], 
    ["q1", "q2", "e"], 
    ["q1", "q0", "a"],
    ["q2", "q2", "b"]
]
afnLambda = AFNLambda(language = language, 
          initial_state = initial_state, 
          valid_states = [2], 
          states = states, 
          delta = delta)

#afn.procesar("aba")
afnLambda.calcularLambdaClausura(current_state = 2)

[2]

In [406]:
afnLambda.procesarCadena("abbaab")

True


In [400]:
afnLambda.procesarCadenaConDetalles("abb")

Procesamientos que terminan en un estado de aceptación: 

Procesamiento No.  0
[('0', 'a'), ('0', 'b'), ('1', 'b'), ('1', 'e'), ('2', '')]
 
Procesamiento No.  1
[('0', 'a'), ('0', 'b'), ('1', 'e'), ('2', 'b'), ('2', '')]
 


In [401]:
afnLambda.procesarListaCadenas(
    ["abb", "abbba", "aaaab"], 
    "resultadosAFNLambda.txt", 
    True
)

------Nueva cadena-----

abb	[['00112', 1, 'abbe'], ['', '', ''], ['', '', ''], 2, 2, 0, 0, 'Si']
('0', 'a')	('0', 'b')	('1', 'b')	('1', 'e')	('2', '')	2	2	0	0	Si	

------Nueva cadena-----

abbba	[['', '', ''], ['001110', -1, 'abbba'], ['001112', 0, 'abbbe'], 4, 0, 3, 1, 'No']
('0', 'a')	('0', 'b')	('1', 'b')	('1', 'b')	('1', 'a')	('0', '')	4	0	3	1	No	

------Nueva cadena-----

aaaab	[['0000012', 1, 'aaaabe'], ['', '', ''], ['', '', ''], 1, 1, 0, 0, 'Si']
('0', 'a')	('0', 'a')	('0', 'a')	('0', 'a')	('0', 'b')	('1', 'e')	('2', '')	1	1	0	0	Si	



'Archivo creado con éxito'

#### 5. Clase Prueba.

In [293]:
class ClasePrueba:
    
    def __init__(self):
        return ""
    
    """
    Invoca a los otros para que puedan ser comentados fácilmente y poder 
    escoger cuál se va a probar.
    """
    def main():
        return ""
    
    """
    Crear autómatas AFD, procesar cadenas con y sin detalles, procesar listas
    de cadenas, generar archivos.
    """
    def probarAFD(self):
        return ""
    
    """
    Crear autómatas AFN, procesar cadenas mostrando solo un procesamiento de
    aceptación, procesar cadenas mostrando toos los porcesamientos posibles, 
    consultar los procesamientos de aceptación, abortados y de rechazo.
    """
    def probarAFN():
        return ""
    
    """
    Crear autómatas AFN-Lambda, calcular la Lambda-clausura de un estado, 
    calcular la Lambda-clausura de un conjunto de estados, procesar cadenas
    mostrando solo un procesamiento de aceptación, procesar cadenas mostrando
    todos los procesamientos posibles, consultar los procesaminetos de 
    aceptación, abortado y de rechazo, procesar listas de cadenas, generar 
    archivos.
    """
    def probarAFNLambda():
        return ""

In [ ]:
   # Get New States Numbers for lambda transition.
            #newStatesNumbersLambda = self.getStatesAccesibleNumbers(
            #    current_state, 
            #    self.language.index("e") 
            #)
            #print(current_state, "new states lamda: ", newStatesNumbersLambda)
            
            #if newStatesNumbersLambda != []:
                    #for newStateLambda in newStatesNumbersLambda:
                        #acceptance_list.append(self.processChain(newStateLambda, chain))
            

In [352]:
# Automata Class.
class Automata: 
    def __init__(self, language = [], initial_state = 0, states = [], valid_states = [], delta = []):
        
        #self.current_state = initial_state
        self.language = language
        self.valid_states = valid_states
        self.adjMatrix = self.generateAdyacencyMatrix(states, delta)
        
    def generateAdyacencyMatrix(self, states, delta):
        
        # Create matrix object.
        adjMatrix = list()
        
        # Number of layers of the adyacency matrix (depth).
        n = len(states)
        for i in range(len(self.language)):
              adjMatrix.append(np.zeros((n, n)))
                
        # Delta codifies the edges between nodes.
        for edge in delta:
            
            # Example: edge = ["q0", "q1", "a"]
            # Get states number.
            first_state = states.index(edge[0])
            second_state = states.index(edge[1])
            
            # Get instruction.
            instruction = self.language.index(edge[2])
            
            # Add new edge.
            adjMatrix[instruction][first_state][second_state] = 1
        
        # Return object.
        return adjMatrix
    
    # Process Chain.
    def processChain(self, current_state, chain):
        
        print("Current State: ", current_state)
        
        # If chain is not null.
        if chain != "":
            
            # Get New States Numbers for new instruction in the chain.
            newStatesNumbers = self.getStatesAccesibleNumbers(
                current_state, 
                self.language.index(chain[0])   
            )
            newStatesNumbersLambda = self.getStatesAccesibleNumbers(
                current_state, 
                self.language.index("e") 
            )
            print("New States Chain: ", newStatesNumbers)
            print("New States Lambda: ", newStatesNumbersLambda)
                    
            # If automata can move to new states.
            #print(newStatesNumbers)
            if newStatesNumbers != []:
                acceptance_list = list()
                for newState in newStatesNumbers:
                    acceptance_list.append(self.processChain(newState, chain[1:]))
                for newLambdaState in newStatesNumbersLambda: 
                    acceptance_list.append(self.processChain(newLambdaState, chain))
                return self.checkAcceptance(acceptance_list)
 
            # Aborted! Since there is no where to move.
            else: 
                return 0    # Aborted.       
        
        # If chain is null.
        else: 
            print("Chain is null!")
            if current_state in self.valid_states:
                return 1    # Accepted.
            else:
                return -1   # Rejected.
    
    # Get functions.
    def getAdyacencyMatrix(self, layer = 0):
        return (self.adjMatrix[layer])
    
    def getValidStates(self):
        return (self.valid_states)
    
    # Utility functions.
    def getStatesAccesibleNumbers(self, current_state, instruction):
        
        layer = self.adjMatrix[instruction]
        bridges = layer[current_state]
        numbers = list()
        for b in range(len(bridges)):
            if bridges[b] == 1: numbers.append(b)
        return numbers
    
    def checkAcceptance(self, acceptanceList):
        if 1 in acceptanceList: return 1
        else: return 0

In [353]:
"""
delta = [
          ["q0", "q0", "a"], 
          ["q0", "q1", "b"], 
          ["q1", "q1", "b"], 
          ["q1", "q0", "a"], 
        ]
delta = [
    ["q0", "q0", "a"], 
    ["q0", "q1", "a"], 
    ["q1", "q1", "b"], 
    ["q1", "q0", "a"], 
    ["q0", "q2", "e"], 
    ["q2", "q2", "a"],
    ["q2", "q1", "b"], 
]
"""
delta = [
    ["q0", "q0", "a"], 
    ["q0", "q1", "e"], 
]

autom = Automata(language = ["a", "b", "e"], 
                 states = ["q0", "q1"], 
                 delta = delta, 
                 valid_states = [1]
                )

In [354]:
autom.processChain(0, "a")

Current State:  0
New States Chain:  [0]
New States Lambda:  [1]
Current State:  0
Chain is null!
Current State:  1
New States Chain:  []
New States Lambda:  []


0

In [336]:
autom.getAdyacencyMatrix(2)

array([[0., 1.],
       [0., 0.]])

In [127]:
autom.getStatesAccesibleNumbers([0, 1, 0, 0, 1])

[1, 4]

In [119]:
autom.checkAcceptance([-1, 0, 0, 0])

0

In [48]:
lista = [0, 1, 0, 1, 0, 0, 1]

In [50]:
lista.index_all(1)

AttributeError: 'list' object has no attribute 'index_all'

In [106]:
chain = "a"

In [107]:
chain[1:]

''